<a href="https://colab.research.google.com/github/peculab/PROGRAM/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai

In [3]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [4]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [5]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1jR3qRQr2ZvWYKNuv8wen_-eTZWdc5a-LLvH7iymn2zw/edit?usp=sharing"
WORKSHEET_NAME = "工作表2"

REQUIRED_COLUMNS = ["日期", "科目", "作業成績"]

_auth_done = False
_gc = None
_ws = None

In [6]:
# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

In [11]:
def get_ai_summary(grades):
    """
    呼叫 Gemini 模型來生成成績摘要與常見迷思。
    """
    model = genai.GenerativeModel('gemini-1.5-flash')

    # 準備給 AI 的提示
    prompt_text = "以下是學生的成績列表，請幫我根據這些成績，產出一個簡單的摘要與常見迷思整理（不評分，只做總結）。\n\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成摘要... ---")
    try:
        response = model.generate_content(prompt_text)
        summary = response.text
        return summary
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 摘要生成失敗。"

In [12]:
new_grades = get_user_grades()

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
已記錄：日期: 2025-09-25, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
已記錄：日期: 2025-09-25, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q


In [13]:
new_grades

[['2025-09-25', 'a', 10], ['2025-09-25', 'b', 20]]

In [14]:
get_ai_summary(new_grades)


--- 正在呼叫 AI 模型生成摘要... ---


'**摘要:**\n\n學生於2025年9月25日參加了科目a和科目b的測驗。科目a的成績為10分，科目b的成績為20分。\n\n\n**常見迷思整理:** (由於資料不足，此部分只能提出可能存在的迷思，而非根據資料推論)\n\n* **迷思一：單次測驗成績能完全反映學生的學習狀況。**  單一測驗成績可能受到多種因素影響，例如考試當天的狀態、題目難度等，並不能完全代表學生的真實學習水平。\n* **迷思二：高分代表完全理解所有知識點。**  即使獲得高分，也可能存在某些知識點理解不夠深入或存在知識漏洞的情況。\n* **迷思三：低分就代表學習完全失敗。** 低分可能反映出學習方法需要調整，或需要更多時間和努力，而非學習的徹底失敗。\n\n\n**需注意:**  以上迷思整理是基於一般學習情況的常見誤解，並非針對學生這兩科的具體情況進行分析。  要更精確的分析，需要更多關於學生的學習過程、學習方法以及考試內容的資訊。\n'

In [16]:
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)





        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()

--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
已記錄：日期: 2025-09-25, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
已記錄：日期: 2025-09-25, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：c
請輸入 c 的成績：30
已記錄：日期: 2025-09-25, 科目: c, 成績: 30

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---

--- 正在呼叫 AI 模型生成摘要... ---

--- AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
**成績摘要:**

2025年9月25日，學生在科目a、b、c的成績分別為10、20和30。  科目c的成績最高，科目a的成績最低。


**常見迷思整理 (基於僅有的資料，可能無法涵蓋所有迷思):**

* **迷思一：單次測驗成績代表整體學習能力:**  僅憑這一次的成績，無法完整評估學生的學習能力和理解程度。  學生在其他方面可能表現突出，而這一次的測驗成績可能受到許多因素影響(例如：當天身體狀況、考試題型等)。

* **迷思二：高分代表完全掌握知識:**  即使科目c取得最高分，也不代表學生完全掌握該科目的所有知識點。  可能只是在這次考試中表現較好。


由於資料有限，以上僅為初步的摘要和常見迷思整理。  若有更多學生的成績資料，可以進行更深入的分析。

--------------------------------------------------
